### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [3]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are several reasons in favor of choosing AutoGen for your new AI Agent project:

1. **Efficiency and Speed**: AutoGen enhances productivity by increasing drafting and processing speeds, allowing teams to create outputs more quickly.

2. **Scalability**: The framework is designed to handle multiple agents, making it suitable for large and complex applications. This scalability ensures that as your project grows, AutoGen can accommodate increased demands.

3. **Multi-Agent Cooperation**: AutoGen enables agents to communicate and collaborate effectively, fostering divergent thinking and improving reasoning and factuality within the outputs.

4. **Ease of Use**: Being an open-source framework, AutoGen simplifies the orchestration and automation of workflows, making it accessible for developers to implement and customize.

5. **Built-In Features**: It includes useful built-in functionalities such as retry logic, model fallback, caching, budgeting, and goal-setting, which enhance functionality and user experience.

6. **Support for Multiple Models**: AutoGen allows for multi-model support, giving developers the flexibility to integrate various AI models as needed.

7. **Focused Applications**: AutoGen has shown success in specialized areas like proposal writing and RFP management, offering high-quality output and compliance.

Overall, the combination of enhanced efficiency, scalability, and collaborative capabilities makes AutoGen a compelling choice for developing sophisticated AI agent applications.

TERMINATE

## Cons of AutoGen:
Here are some cons of using AutoGen for your AI Agent project:

1. **Steep Learning Curve**: Many users find that AutoGen has a challenging learning curve, which can be particularly difficult for individuals who are not technically inclined.

2. **User Difficulty**: The interface and functionalities may not be intuitive, leading to potential frustration and inefficiency among users.

3. **Frequent Changes**: AutoGen may undergo frequent updates and changes, which can complicate the learning process and require users to continually adapt.

4. **Limited documentation**: Depending on the complexity of the tool, there may be inadequate documentation available, making it harder for users to troubleshoot issues or fully utilize its features.

5. **Integration Challenges**: AutoGen might face difficulties when integrating with existing systems or technologies, leading to additional overhead in development and maintenance.

These factors can impact the overall experience and effectiveness of using AutoGen in your project. 

TERMINATE



## Decision:

Given the thorough assessment of both the pros and cons of AutoGen, I recommend proceeding with its use for the project. The advantages of enhanced efficiency, scalability, and multi-agent cooperation present a strong case for its selection, especially in sophisticated AI applications. While there are valid concerns about the learning curve and potential integration challenges, these can be mitigated through proper training and support. The benefits outweigh the drawbacks, positioning AutoGen as a suitable choice for driving project success.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()